In [ ]:
from huggingface_hub import InferenceClient
import json
import os


In [ ]:

client = InferenceClient(
    model="meta-llama/Llama-3.1-8B-Instruct",
    token=os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

In [3]:
response = client.chat_completion(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"},
    ],
    max_tokens=500,
)

print(response.choices[0].message.content)

The capital of France is Paris.


In [8]:
prompt = """
You are an AI model trained to classify text.

I will provide the name of a professional sports team.

You will reply with the sports league in which they compete.
"""

In [10]:
response = client.chat_completion(
    messages=[
        {
            "role": "system",
            "content": prompt,
        },
        {
            "role": "user",
            "content": "Minnesota Vikings",
        }
    ],
)
print(response.choices[0].message.content)

National Football League (NFL)


In [48]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "answer",
        "schema": {
            "type": "object",
            "properties": {
                "answer": {
                    "type": "string",
                    "enum": [
                        "Major League Baseball (MLB)",
                        "National Football League (NFL)",
                        "National Basketball Association (NBA)",
                        "Other or Unknown",
                    ],
                    "description": "Select one of the major professional sports leagues.",
                }
            },
            "required": ["answer"],
            "additionalProperties": False
        },
        "strict": True
    }
}

In [58]:
def classify_team(name):
    prompt = """
You are an AI model trained to classify text.

I will provide the name of a professional sports team.

You will reply with the sports league in which they compete.
"""

    response = client.chat_completion(
        messages=[
            {
                "role": "system",
                "content": prompt,
            },
           {
                "role": "user",
                "content": "Los Angeles Rams",
            },
            {
                "role": "assistant",
                "content": "National Football League (NFL)",
            },
            {
                "role": "user",
                "content": "Los Angeles Dodgers",
            },
            {
                "role": "assistant",
                "content": " Major League Baseball (MLB)",
            },
            {
                "role": "user",
                "content": "Los Angeles Lakers",
            },
            {
                "role": "assistant",
                "content": "National Basketball Association (NBA)",
            },
            {
                "role": "user",
                "content": "Los Angeles Kings",
            },
            {
                "role": "assistant",
                "content": "Other",
            },
            {
                "role": "user",
                "content": name,
            },
        ],
        response_format=response_format,
        temperature=0,
    )

    raw_content = response.choices[0].message.content
    if isinstance(raw_content, str):
        try:
            return json.loads(raw_content).get("answer", raw_content)
        except json.JSONDecodeError:

            return raw_content
    return raw_content


In [59]:
team_list = ["Minnesota Twins", "Minnesota Vikings", "Minnesota Timberwolves"]

In [60]:
for team in team_list:
    league = classify_team(team)
    print([team, league])

['Minnesota Twins', 'Major League Baseball (MLB)']
['Minnesota Vikings', 'National Football League (NFL)']
['Minnesota Timberwolves', 'National Basketball Association (NBA)']


In [61]:
classify_team("Minnesota Golden Gophers")

'Other or Unknown'